In [2]:
#!pip install twilio,python-dotenv

In [6]:
import pandas as pd
import os

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

from twilio.rest import Client
import time


In [27]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="Config.env")

twilio_acc = os.getenv('TWILIO_ACC')
twilio_secret = os.getenv('TWILIO_SECRET')
twilio_phone = os.getenv('TWILIO_PHONE')
WEATHER_API = os.getenv('WEATHER_API')

# Api call 

In [108]:
query = 'Tokyo'
url = "http://api.weatherapi.com/v1/forecast.json?key=" + WEATHER_API + "&q=" + query + "&days=1&aqi=no&alerts=no"

In [109]:
response = requests.get(url).json()

# Extract for all day

In [110]:
def get_forecast(response,i):
    general = response['forecast']['forecastday'][0]['hour'][i]
    fecha= general['time'].split()[0]
    hour = general['time'].split()[1].split(":")[0]
    condition = general["condition"]['text']
    temp_C = general["temp_c"]
    will_rain = general["will_it_rain"]
    chance_of_rain = general['chance_of_rain']
    
    return fecha,hour,condition,temp_C,will_rain,chance_of_rain
    

In [111]:
data = []

for x in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour="green"):
    data.append(get_forecast(response,x))
data

100%|███████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 47798.34it/s]


[('2023-08-10', '00', 'Patchy rain possible', 27.2, 1, 77),
 ('2023-08-10', '01', 'Patchy rain possible', 26.9, 1, 78),
 ('2023-08-10', '02', 'Patchy rain possible', 26.8, 1, 96),
 ('2023-08-10', '03', 'Patchy rain possible', 26.6, 1, 96),
 ('2023-08-10', '04', 'Patchy rain possible', 26.7, 1, 96),
 ('2023-08-10', '05', 'Patchy rain possible', 26.6, 1, 73),
 ('2023-08-10', '06', 'Patchy rain possible', 26.7, 1, 73),
 ('2023-08-10', '07', 'Partly cloudy', 27.6, 0, 0),
 ('2023-08-10', '08', 'Light rain shower', 28.7, 1, 74),
 ('2023-08-10', '09', 'Partly cloudy', 29.4, 0, 0),
 ('2023-08-10', '10', 'Partly cloudy', 29.5, 0, 0),
 ('2023-08-10', '11', 'Cloudy', 29.8, 0, 0),
 ('2023-08-10', '12', 'Patchy rain possible', 29.9, 1, 88),
 ('2023-08-10', '13', 'Partly cloudy', 31.1, 0, 0),
 ('2023-08-10', '14', 'Partly cloudy', 31.8, 0, 0),
 ('2023-08-10', '15', 'Partly cloudy', 32.5, 0, 0),
 ('2023-08-10', '16', 'Sunny', 32.4, 0, 0),
 ('2023-08-10', '17', 'Sunny', 32.0, 0, 0),
 ('2023-08-10', '1

In [112]:
col = ["fecha","hour","condition","temp_C","will_rain","chance_of_rain"]
df = pd.DataFrame(data,columns=col)
df

,fecha,hour,condition,temp_C,will_rain,chance_of_rain
0,2023-08-10,00,Patchy rain possible,27.2,1,77
1,2023-08-10,01,Patchy rain possible,26.9,1,78
2,2023-08-10,02,Patchy rain possible,26.8,1,96
3,2023-08-10,03,Patchy rain possible,26.6,1,96
4,2023-08-10,04,Patchy rain possible,26.7,1,96
5,2023-08-10,05,Patchy rain possible,26.6,1,73
6,2023-08-10,06,Patchy rain possible,26.7,1,73
7,2023-08-10,07,Partly cloudy,27.6,0,0
8,2023-08-10,08,Light rain shower,28.7,1,74
9,2023-08-10,09,Partly cloudy,29.4,0,0


In [113]:
df_rain = df[df["will_rain"] == 1]
df_rain = df_rain[["hour","condition"]]
df_rain.set_index('hour',inplace=True)

In [114]:
df_rain

,condition
hour,
00,Patchy rain possible
01,Patchy rain possible
02,Patchy rain possible
03,Patchy rain possible
04,Patchy rain possible
05,Patchy rain possible
06,Patchy rain possible
08,Light rain shower
12,Patchy rain possible


# Message

In [118]:
message = "\n Hola el pronostico del tiempo de lluvia hoy " + df['fecha'][0] + ' en ' + query + ' es \n\n\n' + str(df_rain)

In [119]:
message

'\n Hola el pronostico del tiempo de lluvia hoy 2023-08-10 en Tokyo es \n\n\n                 condition\nhour                      \n00    Patchy rain possible\n01    Patchy rain possible\n02    Patchy rain possible\n03    Patchy rain possible\n04    Patchy rain possible\n05    Patchy rain possible\n06    Patchy rain possible\n08       Light rain shower\n12    Patchy rain possible'

In [120]:
client = Client(twilio_acc, twilio_secret)

message = client.messages \
    .create(
         body=message,
         from_=twilio_phone,
         to='+528120075281'
     )
print(message.sid)


SMd9baa829eb82199fa58dfaa86c3d33a3
